In [5]:
# header info
import pandas as pd
import geopandas as gpd
from siuba import *
import numpy as np
from shared_utils import utils
pd.options.display.max_columns = 100
import gcsfs
from calitp_data_analysis.sql import to_snakecase

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_prioritization/"

# CSIS Data Entry Scoring - Safety

## Part 1: Evaluate Crash Reduction Factors

In [10]:
safety_data = pd.read_parquet(f'{GCS_FILE_PATH}data_entry_raw_safety.parquet')

In [4]:
safety_data.head()

submission_log_number program  \
0                      1    SCCP   
1                      2    SCCP   
2                      3    SCCP   
3                      4    SCCP   
4                      5    SCCP   

                                        project_name data_enterer_name  \
0  U.S. 101 Connected Communities Corridor Rail a...       Llisel Ayon   
1  Watsonville-Santa Cruz Multimodal Corridor Pro...       Llisel Ayon   
2        SMART Windsor Rail System Extension Project       Llisel Ayon   
3  Metrolink Lilac to Sycamore Avenue Double Trac...       Llisel Ayon   
4          Inglewood Transit Connector (ITC) Project       Nick Compin   

  done_y_n                                              notes  \
0        Y  The majority of these countermeasures will be ...   
1        Y                                               None   
2        Y                                               None   
3        Y  Double Track Rail. With new platform. No PSCMS...   
4        Y  Existing lane width is critical to impact 11' ...   

                countermeasure_1  crf_1  \
0  Bicycle Lanes, Mode: Bike/Ped   0.30   
1      Bicycle Lanes, Mode: Bike   0.49   
2      Bicycle Lanes, Mode: Bike   0.30   
3                           None    NaN   
4                     Bike lanes   0.27   

                                         reference_1  \
0  https://highways.dot.gov/safety/proven-safety-...   
1  https://highways.dot.gov/safety/proven-safety-...   
2  https://highways.dot.gov/safety/proven-safety-...   
3                                               None   
4  https://www.cmfclearinghouse.org/study_detail....   

                                    countermeasure_2  crf_2  \
0  Rectangular Rapid Flashing Beacons, Mode: Bike...   0.47   
1        Walkways (sidewalk gap closures), Mode: Ped   0.65   
2                                               None    NaN   
3                                               None    NaN   
4                                               None    NaN   

                                         reference_2  \
0  https://highways.dot.gov/safety/proven-safety-...   
1  https://highways.dot.gov/safety/proven-safety-...   
2                                               None   
3                                               None   
4                                               None   

                                    countermeasure_3  crf_3  \
0  Pedestrain Hybrid Beacons (HAWK signals), Mode...   0.29   
1      Rectangular Rapid Flashing Beacons, Mode: Ped   0.47   
2                                               None    NaN   
3                                               None    NaN   
4                                               None    NaN   

                                         reference_3     countermeasure_4  \
0  https://highways.dot.gov/safety/proven-safety-...  Walkways, Mode: Ped   
1  https://highways.dot.gov/safety/proven-safety-...                 None   
2                                               None                 None   
3                                               None                 None   
4                                               None                 None   

   crf_4                                        reference_4 countermeasure_5  \
0   0.65  https://highways.dot.gov/safety/proven-safety-...             None   
1    NaN                                               None             None   
2    NaN                                               None             None   
3    NaN                                               None             None   
4    NaN                                               None             None   

   crf_5 reference_5 countermeasure_6  crf_6 reference_6  
0    NaN        None             None    NaN        None  
1    NaN        None             None    NaN        None  
2    NaN        None             None    NaN        None  
3    NaN        None             None    NaN        None  
4    NaN        None

In [11]:
safety_data >> count(_.done_y_n)

done_y_n   n
0        Y  53
1      NaN  27

In [13]:
# count scored projects
safety_data >> count(_.done_y_n,_.crf_1)

done_y_n  crf_1   n
0         Y  0.008   1
1         Y  0.130   1
2         Y  0.140   1
3         Y  0.200   4
4         Y  0.250   1
5         Y  0.270   1
6         Y  0.280   1
7         Y  0.290   1
8         Y  0.300   4
9         Y  0.320   1
10        Y  0.400   2
11        Y  0.410   1
12        Y  0.430   2
13        Y  0.450   1
14        Y  0.490   5
15        Y  0.500   1
16        Y  0.710   2
17        Y  0.970   1
18        Y    NaN  22
19      NaN    NaN  27

In [15]:
# replace missing CRFs with 0 for math
safety_data[["crf_1","crf_2","crf_3","crf_4","crf_5","crf_6"]] = safety_data[["crf_1","crf_2","crf_3","crf_4","crf_5","crf_6"]].fillna(0)

Reference: https://www.cmfclearinghouse.org/collateral/Combining_Multiple_CMFs_Final.pdf 
CCRFi = 1 – [(1-CRF1i)*(1-CRF2i)*(1-CRF3i)] 

In [19]:
# combine crfs
safety_data = (safety_data
              >> mutate(crf_combined = 1-((1-_.crf_1)*(1-_.crf_2)*(1-_.crf_3)*(1-_.crf_4)*(1-_.crf_5)*(1-_.crf_6)),
                        crf_cat = case_when({_.crf_combined>=0.1 : "High",
                                             _.crf_combined==0 : "None",
                                             _.crf_combined<0.1 : "Low"
                                            })
                       )
              )

In [21]:
safety_data >> count(_.done_y_n,_.crf_cat, _.crf_combined)

done_y_n crf_cat  crf_combined   n
0         Y    High      0.130000   1
1         Y    High      0.140000   1
2         Y    High      0.200000   3
3         Y    High      0.270000   1
4         Y    High      0.300000   2
5         Y    High      0.430000   2
6         Y    High      0.438400   1
7         Y    High      0.450000   1
8         Y    High      0.480800   1
9         Y    High      0.486080   1
10        Y    High      0.556300   1
11        Y    High      0.580000   1
12        Y    High      0.636571   1
13        Y    High      0.679150   1
14        Y    High      0.720000   1
15        Y    High      0.779680   1
16        Y    High      0.794100   1
17        Y    High      0.799900   1
18        Y    High      0.821500   2
19        Y    High      0.860266   1
20        Y    High      0.861256   1
21        Y    High      0.905395   1
22        Y    High      0.907806   1
23        Y    High      0.919699   1
24        Y    High      0.955921   1
25        Y    High      0.970000   1
26        Y    None      0.000000  22
27      NaN    None      0.000000  27

In [22]:
safety_data >> filter(_.crf_combined>0.9)

submission_log_number program  \
0                       1    SCCP   
1                       2    SCCP   
8                      10    SCCP   
30                     18    TCEP   
37                     20    SCCP   

                                         project_name data_enterer_name  \
0   U.S. 101 Connected Communities Corridor Rail a...       Llisel Ayon   
1   Watsonville-Santa Cruz Multimodal Corridor Pro...       Llisel Ayon   
8        East Bay Greenway Multimodal Project Phase 1       Llisel Ayon   
30                      Konocti Corridor - Segment 2B       Nick Compin   
37     Silicon Valley Express Lanes Program – Phase 5       Llisel Ayon   

   done_y_n                                              notes  \
0         Y  The majority of these countermeasures will be ...   
1         Y                                               None   
8         Y  Additional countermeasures include walkways to...   
30        Y                                               None   
37        Y             Reduction is for cross-median crashes.   

                                countermeasure_1  crf_1  \
0                  Bicycle Lanes, Mode: Bike/Ped   0.30   
1                      Bicycle Lanes, Mode: Bike   0.49   
8   Crosswalk Visibility Enhancements, Mode: Ped   0.40   
30                            Convert 2 - 4 lane   0.29   
37                   Median Barriers, Mode: Auto   0.97   

                                          reference_1  \
0   https://highways.dot.gov/safety/proven-safety-...   
1   https://highways.dot.gov/safety/proven-safety-...   
8   https://highways.dot.gov/safety/proven-safety-...   
30  https://www.cmfclearinghouse.org/study_detail....   
37  https://highways.dot.gov/safety/proven-safety-...   

                                     countermeasure_2  crf_2  \
0   Rectangular Rapid Flashing Beacons, Mode: Bike...   0.47   
1         Walkways (sidewalk gap closures), Mode: Ped   0.65   
8                           Bicycle Lanes, Mode: Bike   0.49   
30                                          shoulders   0.87   
37                                               None   0.00   

                                          reference_2  \
0   https://highways.dot.gov/safety/proven-safety-...   
1   https://highways.dot.gov/safety/proven-safety-...   
8   https://highways.dot.gov/safety/proven-safety-...   
30  https://www.cmfclearinghouse.org/study_detail....   
37                                               None   

                                     countermeasure_3  crf_3  \
0   Pedestrain Hybrid Beacons (HAWK signals), Mode...   0.29   
1       Rectangular Rapid Flashing Beacons, Mode: Ped   0.47   
8       Rectangular Rapid Flashing Beacons, Mode: Ped   0.47   
30                               remove fixed objects   0.13   
37                                               None   0.00   

                                          reference_3  \
0   https://highways.dot.gov/safety/proven-safety-...   
1   https://highways.dot.gov/safety/proven-safety-...   
8   https://highways.dot.gov/safety/proven-safety-...   
30  https://www.cmfclearinghouse.org/study_detail....   
37                                               None   

                          countermeasure_4  crf_4  \
0                      Walkways, Mode: Ped   0.65   
1                                     None   0.00   
8   Leading Pedestrian Interval, Mode: Ped   0.13   
30                                    None   0.00   
37                                    None   0.00   

                                          reference_4  \
0   https://highways.dot.gov/safety/proven-safety-...   
1                                                None   
8   https://highways.dot.gov/safety/proven-safety-...   
30                                               None   
37                                               None   

                                     countermeasure_5  crf_5  \
0                                                None

In [23]:
safety_data >> filter(_.crf_combined<0.2, _.crf_combined>0)

submission_log_number program  \
5                       7    SCCP   
6                       8    SCCP   
31                     19    TCEP   
35                     23    TCEP   
49                     47    TCEP   

                                         project_name data_enterer_name  \
5                                   I-5 Managed Lanes       Nick Compin   
6                                  Bay Skyway Phase 1       Nick Compin   
31                              Fix 5 Cascade Gateway       Nick Compin   
35         I-710 Integrated Corridor Management (ICM)       Llisel Ayon   
49  Interstate 10 Corridor Freight and Managed Lan...       Nick Compin   

   done_y_n                                     notes  \
5         Y        Impact on Free, HOT, or All Lanes?   
6         Y                                      None   
31        Y  Onramp/truck climbing no exact PSCM CRFs   
35        Y                                      None   
49        Y                                      None   

               countermeasure_1  crf_1  \
5            Convert HOV to HOT   0.20   
6                Multi-use path   0.14   
31              auxiliary lanes   0.20   
35  Leading Pedestrian Interval   0.13   
49                    Aux Lanes   0.20   

                                          reference_1 countermeasure_2  crf_2  \
5   https://www.cmfclearinghouse.org/study_detail....             None    0.0   
6   https://www.cmfclearinghouse.org/study_detail....             None    0.0   
31  https://www.cmfclearinghouse.org/study_detail....             None    0.0   
35  https://highways.dot.gov/safety/proven-safety-...             None    0.0   
49  https://www.cmfclearinghouse.org/study_detail....             None    0.0   

   reference_2 countermeasure_3  crf_3 reference_3 countermeasure_4  crf_4  \
5         None             None    0.0        None             None    0.0   
6         None             None    0.0        None             None    0.0   
31        None             None    0.0        None             None    0.0   
35        None             None    0.0        None             None    0.0   
49        None             None    0.0        None             None    0.0   

   reference_4 countermeasure_5  crf_5 reference_5 countermeasure_6  crf_6  \
5         None             None    0.0        None             None    0.0   
6         None             None    0.0        None             None    0.0   
31        None             None    0.0        None             None    0.0   
35        None             None    0.0        None             None    0.0   
49        None             None    0.0        None             None    0.0   

   reference_6  crf_combined crf_cat  
5         None          0.20    High  
6         None          0.14    High  
31        None          0.20    High  
35        None          0.13    High  
49        None          0.20    High

## Part 2: Evaluate Safety Need

In [25]:
project_geo = gpd.read_parquet(f'{GCS_FILE_PATH}Survey123_Geo/cleaned_survey123_sample13.parquet')

In [26]:
project_geo.explore()